In [5]:
"""
Загрузка данных
"""
import re
import os
import pandas as pd
import pyodbc
import datetime
from datetime import datetime

pd.set_option('max_colwidth', 160)

conn_str = r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};"\
    r"DBQ=C:\Users\gdoku\YandexDisk\Документы\Семейные финансы\FF.accdb;"


def new_transactions(syn_acc, anal_acc, transactions):
    """Подготовка таблицы для загрузки."""

    conn = pyodbc.connect(conn_str)
    if syn_acc == 661 and anal_acc == 1:
        SQL = "SELECT [003_Факты].Дата,[003_Факты].СобНаим, [004_Проводки].Оборот \
                FROM 003_Факты INNER JOIN 004_Проводки ON [003_Факты].ФактНом = [004_Проводки].ФактНом \
                WHERE ([004_Проводки].СинтСчётНом={} AND [004_Проводки].АналСчётНом={}) \
                OR ([004_Проводки].СинтСчётНом=661 AND [004_Проводки].АналСчётНом=3)"\
                .format(syn_acc, anal_acc)
    else:
        SQL = "SELECT [003_Факты].Дата,[003_Факты].СобНаим, [004_Проводки].Оборот \
                FROM 003_Факты INNER JOIN 004_Проводки ON [003_Факты].ФактНом = [004_Проводки].ФактНом \
                WHERE [004_Проводки].СинтСчётНом={} AND [004_Проводки].АналСчётНом={}"\
                .format(syn_acc, anal_acc)
    base = pd.read_sql(SQL, conn)
    df = pd.merge(transactions, base, how='left',
                  left_on=['date', 'amount'],
                  right_on=['Дата', 'Оборот'])
    new_transactions = df[df['Оборот'].isna()].iloc[:, :3]
    new_transactions = new_transactions[new_transactions['date']
                                        >= '2020-04-01']
    conn.close()

    return new_transactions


def upload_to_base(syn_acc, anal_acc, transactions):
    """Функция загрузки новых проводок в базу."""

    df = new_transactions(syn_acc, anal_acc, transactions)

    # Определение типа операций
    conn = pyodbc.connect(conn_str)
    SQL = 'SELECT ID, Description, Name, ds, da, cs, ca \
        FROM 008_LEARNING \
        WHERE ds={} AND da={}'.format(syn_acc, anal_acc)
    base = pd.read_sql(SQL, conn)

    # Проведение прододок
    for i, base_row in base.iterrows():
        lookup = base_row['Description']
        post = base_row['Name']
        df1 = df[df['description'].str.contains(lookup, case = False) == True]
        for index, row in df1.iterrows():
            with conn.cursor() as crsr:
                crsr.execute("INSERT INTO 003_Факты ( Дата, СобНаим ) VALUES(?, ?)",
                             row['date'], post)
                post_id = crsr.execute("SELECT Max(ФактНом) FROM 003_Факты WHERE СобНаим = ?",
                                       post).fetchall()[0][0]
                crsr.execute('INSERT INTO 004_Проводки VALUES({}, {}, {}, {}, {}, 0)'.
                             format(post_id, 0, syn_acc, anal_acc, row['amount']))
                crsr.execute('INSERT INTO 004_Проводки VALUES({}, {}, {}, {}, {}, 0)'.
                             format(post_id, 1, base_row['cs'], base_row['ca'], -row['amount']))
    conn.close()


def vtb_bank_details(path):
    directory = r'C:/Users/gdoku/YandexDisk/Документы/Договоры/Банк ВТБ (ПАО)/'
    files = os.listdir(os.path.join(directory, path))
    counter = 0
    for file in files:
        data = pd.read_csv(os.path.join(directory, path, file),
                           sep=';', encoding='cp1251', header=6)
        if counter == 0:
            transactions = data.loc[data['Статус'] == 'Исполнено']
            counter += 1
        else:
            transactions = transactions.append(data.loc[data['Статус'] == 'Исполнено'],
                                               ignore_index=True)
    transactions = transactions.drop_duplicates()
    transactions = transactions.iloc[:, [1, 5, 7]]
    transactions.columns = ['date', 'amount', 'description']
    transactions['date'] = pd.to_datetime(pd.to_datetime(transactions['date'],
                                                         format='%Y-%m-%d').dt.date)
    transactions['amount'] = [float(x.replace(',', '.').replace(' ', ''))
                              for x in transactions['amount']]
    transactions['description'] = transactions['description'].fillna('Пропуск')

    return transactions


def sberbank_details(path):
    directory = r'C:/Users/gdoku/YandexDisk/Документы/Договоры/Сбербанк/'
    files = os.listdir(os.path.join(directory, path))
    counter = 0
    for file in files:
        data = pd.read_html(os.path.join(directory, path, file))[6]
        if counter == 0:
            transactions = data.loc[data['ДАТА'] != 'Вчера']
            counter += 1
        else:
            transactions = transactions.append(data.loc[data['ДАТА'] != 'Вчера'],
                                               ignore_index=True)
    transactions = transactions.drop_duplicates()
    transactions = transactions[transactions['ОПЕРАЦИЯ'].str.contains(
        'Перевод между своими счетами и картами') != True]
    transactions = transactions[transactions['ОПЕРАЦИЯ'].str.contains(
        'Показать по') != True]
    transactions = transactions[transactions['ОПЕРАЦИЯ'].str.contains(
        'сполнен') == True]
    transactions = transactions.dropna(subset=['ПОЛУЧАТЕЛЬ'])
    transactions = transactions.dropna(subset=['СУММА']).iloc[:, :-2]
    transactions['ОПЕРАЦИЯ'] = transactions['ОПЕРАЦИЯ'].map(lambda x: x[:-9])
    transactions['СЧЕТ СПИСАНИЯ'] = transactions['СЧЕТ СПИСАНИЯ'].map(
        lambda x: str(x)[-5:])
    transactions['description'] = transactions['ПОЛУЧАТЕЛЬ'] + ' ' +\
        transactions['ОПЕРАЦИЯ']
    transactions.description = transactions.description.apply(lambda x: x.replace('•••• ', '').replace('.', ''))
    transactions['date'] = pd.to_datetime(
        transactions['ДАТА']+str(datetime.now().year), format='%d.%m%Y')
    transactions['СУММА'] = transactions['СУММА'].map(lambda x: str(x)[:-5])
    transactions['СУММА'] = [float(x.replace(',', '.').replace(' ', ''))
                             for x in transactions['СУММА']]
    transactions['amount'] = transactions['СУММА']
    return transactions.iloc[:, -3:].sort_values(by=['date'])


def ip_vtb_bank_details(path):
    directory = r'C:/Users/gdoku/YandexDisk/Документы/Предпринимательская деятельность/Договор банковского счёта\/'
    files = os.listdir(os.path.join(directory, path))
    counter = 0
    for file in files:
        data = pd.read_csv(os.path.join(directory, path, file),
                           sep=';', encoding='cp1251', header=0)
        if counter == 0:
            transactions = data.iloc[2:]
            counter += 1
        else:
            transactions = transactions.append(data.iloc[2:],
                                               ignore_index=True)
    transactions = transactions.drop_duplicates()
    transactions['Основание платежа'] = transactions['Счет Получателя'] + \
        ' '+transactions['Основание платежа']
    transactions = transactions.iloc[:, [1, 4, 6]]
    transactions.columns = ['date', 'amount', 'description']
    transactions['date'] = pd.to_datetime(pd.to_datetime(transactions['date'],
                                                         format='%d.%m.%Y').dt.date)
    transactions['amount'] = [float(x.replace(',', '.').replace(' ', ''))
                              for x in transactions['amount']]
    transactions['description'] = transactions['description'].fillna('Пропуск')

    return transactions


def upload_data(syn_acc, anal_acc, transactions):
    upload_to_base(syn_acc, anal_acc, transactions)
    display(new_transactions(syn_acc, anal_acc, transactions))


# Загрузка списка операций по мастер-счёту
path = r'Георгий/Договор комплексного обслуживания/ГМС'
transactions = vtb_bank_details(path)
upload_data(510, 6, transactions)

# Загрузка списка операций по мастер-счёту Алены
path = r'Алена/МС'
transactions = vtb_bank_details(path)
upload_data(510, 7, transactions)

# Загрузка списка операций по накопительному счёту
path = r'Георгий/Договор комплексного обслуживания/Копилка'
transactions = vtb_bank_details(path)
upload_data(510, 12, transactions)

# Загрузка списка операций по накопительному счёту Алены
path = r'Алена/Копилка'
transactions = vtb_bank_details(path)
upload_data(510, 9, transactions)

# Загрузка списка операций по кредитной карте
path = r'Георгий/Договор комплексного обслуживания/КК'
transactions = vtb_bank_details(path)
upload_data(661, 1, transactions)

# Загрузка списка операций по Сбербанку Егора
path = r'Егор/'
transactions = sberbank_details(path)
upload_data(510, 11, transactions)

# Загрузка списка операций предпринимательскому счёту
path = r'Выписки/'
transactions = ip_vtb_bank_details(path)
upload_data(510, 1, transactions)

# Загрузка списка операций по Сбербанку Алены
path = r'Алена/'
transactions = sberbank_details(path)
upload_data(510, 5, transactions)

,date,amount,description


,date,amount,description


,date,amount,description


,date,amount,description


,date,amount,description


,description,date,amount


,date,amount,description


,description,date,amount


In [10]:
import pandas as pd
import xlrd
import pyodbc
import datetime
from datetime import datetime, timedelta

def round_date(s):
    return datetime.date(s.year, s.month, s.day)

conn_str = r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};"r"DBQ=C:\Users\gdoku\YandexDisk\Документы\Семейные финансы\FF.accdb;"

# Подготовка датафрейма для загрузки
data = {
    'name': 'Обслуживание расчётного счёта',
    'date': '30.09.2020',
    'amount': 1020,
    'ds': 932,
    'da': 11,
    'dq': 'Null',
    'cs': 600,
    'ca': 21,
    'cq': 'Null'
}

#Загрузка данных
data['date'] = datetime.strptime(data['date'], '%d.%m.%Y')
conn = pyodbc.connect(conn_str)
with conn.cursor() as crsr:
    crsr.execute("INSERT INTO 003_Факты ( Дата, СобНаим ) VALUES(?, ?)", data['date'], data['name'])
    post_id = crsr.execute("SELECT Max(ФактНом) FROM 003_Факты WHERE СобНаим = ?", data['name']).fetchall()[0][0]
    crsr.execute('INSERT INTO 004_Проводки VALUES({}, {}, {}, {}, {}, {})'.format(post_id, 0, data['ds'], data['da'],
                                                                                 data['amount'], data['dq']))
    crsr.execute('INSERT INTO 004_Проводки VALUES({}, {}, {}, {}, {}, {})'.format(post_id, 1, data['cs'], data['ca'],
                                                                                -data['amount'], data['cq']))
conn.close()